In [1]:
from multiprocessing.dummy import active_children
import os
from re import I
import sys
import logging
import argparse

import json
import pandas as pd
import numpy as np

from keras import activations
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import BatchNormalization
from keras.layers import Lambda, Concatenate, Reshape, Add, Dropout
from keras.layers import MultiHeadAttention
from keras.models import Model 

from keras.utils.vis_utils import plot_model

from myattention import Attention
from layers import Aggregate
from load import load_data, load_cla_data

In [24]:
params = {
    "feature_dim": 11,
    "ticker_size": 2,
    "sample_size": 5,
}

time_steps=3
units=11

index = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],
                   [[1,2,3,4,5,6,7,8,9,10,11],
                   [2,3,4,5,6,7,8,9,10,11,12],
                   [3,4,5,6,7,8,9,10,11,12,13]],])
stock1 = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],])
stock2 = np.array([[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],
                [[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]],])
stocks = [stock1, stock2]
y = np.array([14,15,16,17,18])

def time_axis_attention(model_input, feature_dim, units):
    x = Dense(feature_dim, activation='tanh')(model_input)
    x = LSTM(units, return_sequences=True)(x)
    x = Attention(feature_dim)(x)
    x = BatchNormalization()(x)
    return x

context_aggs = []
index_input = Input(shape=(time_steps, params['feature_dim']))
stock_input = Input(shape=(time_steps, params['feature_dim']))
index_model = time_axis_attention(index_input, params['feature_dim'], units)
stock_model = time_axis_attention(stock_input, params['feature_dim'], units)
context_agg = Aggregate(11)([index_model, stock_model])
model = Model(inputs=[index_input, stock_input], outputs=context_agg)
model.compile(loss='mae', optimizer='adam')
plot_model(model, to_file='test.png', show_shapes=True, show_layer_names=True)
weights = None


test = np.array([[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]])

for i, stock in enumerate(stocks[:1]):
    model.fit(x=[index,stock2], y=y, epochs=5)
    weights = model.get_layer('aggregate').get_weights()
    pred = model.predict([test,test]])



Epoch 1/5


2022-07-27 16:16:17.163344: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 2600 num_cores: 12 environment { key: "cpu_instruction_set" value: "SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 262144 l3_cache_size: 12582912 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2022-07-27 16:16:17.163475: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "GenuineIntel" model: "110" frequency: 2600 num_cores: 12 environment { key: "cpu_instruction_set" value: "SSE, SSE2,

1/1 [==============================] - 4s 4s/step - loss: 16.0000
Epoch 2/5
1/1 [==============================] - 0s 5ms/step - loss: 15.9988
Epoch 3/5
1/1 [==============================] - 0s 5ms/step - loss: 15.9977
Epoch 4/5
1/1 [==============================] - 0s 5ms/step - loss: 15.9965
Epoch 5/5
1/1 [==============================] - 0s 6ms/step - loss: 15.9953


ValueError: in user code:

    File "/Users/narijeong/opt/anaconda3/envs/ml/lib/python3.9/site-packages/keras/engine/training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "/Users/narijeong/opt/anaconda3/envs/ml/lib/python3.9/site-packages/keras/engine/training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/narijeong/opt/anaconda3/envs/ml/lib/python3.9/site-packages/keras/engine/training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/narijeong/opt/anaconda3/envs/ml/lib/python3.9/site-packages/keras/engine/training.py", line 1791, in predict_step
        return self(x, training=False)
    File "/Users/narijeong/opt/anaconda3/envs/ml/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/narijeong/opt/anaconda3/envs/ml

In [26]:
[[1,2,3,4,5,6,7,8,9,10,11],
                [2,3,4,5,6,7,8,9,10,11,12],
                [3,4,5,6,7,8,9,10,11,12,13]]

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11],
 [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]

In [16]:
len(weights[0])

11

In [20]:
len(weights[0][0])

11